# Define organoid detection configuration

This notebook allows  and quantifies organoid size. It uses the model trained for [OrgaQuant](https://github.com/TKassis/OrgaQuant). The notebook contains several steps, please start with step 1.


## 1. Start  
- Run the complete notebook (Cell -> Run All)
- Continue with step 2

In [10]:
import os
if os.getcwd().endswith('scripts'):
    path_parent = os.path.dirname(os.getcwd())
    os.chdir(path_parent)

In [11]:
with open('data.conf', 'r') as file:
    image_dir = file.read().replace('\n', '')
with open('models.conf', 'r') as file:
    model_dir = file.read().replace('\n', '')

In [12]:
# Supress General warnings
import warnings
warnings.filterwarnings('ignore')

#Import libraries
import os
import re
import json
import ipywidgets as widgets
from pathlib import Path
import cv2
import tensorflow as tf
import keras
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image, adjust_contrast
from keras_retinanet.utils.visualization import draw_box
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shutil

# Set tensorflow logging
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print(tf.__version__)
print('Dependecies succesfull loaded!')

2.3.0
Dependecies succesfull loaded!


## 2. Setting parameters for organoid detection

Adjust the sliders and press process to assess image. If output is not as required, change settings and try again. Initial values are manually optimized to minimize false positives. Continue to step 3 if you are happy with the result.

Tip: test the same settings with multiple images (controlled by the seed).



In [13]:
slider_image_size = widgets.IntSlider(value=1024, 
                                    min=512,
                                    max=2048,
                                    step=128,
                                    description='Image Size:',
                                    description_tooltip='Larger "Image Size" allows you to detect smaller orgaoids at the cost of computational demand.')

slider_contrast = widgets.FloatSlider(value=2,
                                      min=1,
                                      max=3,
                                      step=0.25,
                                      description='Contrast', 
                                      description_tooltip='Larger "Contrast" can improve detection sometimes.')

slider_threshold  = widgets.FloatSlider(value=0.9,
                                        min=0,
                                        max=1,
                                        step=0.05,
                                        description='Threshold', 
                                        description_tooltip='Use larger "Threshold" to eliminate false positives.')

slider_seed = widgets.IntSlider(value=1,
                                min=1,
                                max=40,
                                step=1,
                                description='Seed',
                                description_tooltip='Change value if parameter optimization should be performed on another image')

dropdown_images = widgets.Dropdown(options=[x.name for x in Path(image_dir).iterdir() if x.is_dir()],
                                 description='Image directory:',
                                 disabled=False)

dropdown_model = widgets.Dropdown(options=[x.name for x in Path(model_dir).glob('**/*.h5')],
                                  description='Model:',
                                  disabled=False)

display(dropdown_images, dropdown_model, slider_image_size, slider_contrast, slider_threshold, slider_seed)

Dropdown(description='Image directory:', options=('test1', 'test2'), value='test1')

Dropdown(description='Model:', options=('orgaquant_intestinal_v3.h5',), value='orgaquant_intestinal_v3.h5')

IntSlider(value=1024, description='Image Size:', description_tooltip='Larger "Image Size" allows you to detect…

FloatSlider(value=2.0, description='Contrast', description_tooltip='Larger "Contrast" can improve detection so…

FloatSlider(value=0.9, description='Threshold', description_tooltip='Use larger "Threshold" to eliminate false…

IntSlider(value=1, description='Seed', description_tooltip='Change value if parameter optimization should be p…

In [14]:
process_button = widgets.Button(
    description='Process ',
    disabled=False,
    button_style='primary',
    tooltip='Click to test configuration',
    icon='cogs' # (FontAwesome names without the `fa-` prefix)
)

outA = widgets.Output()

def detect_organoids():
    print('Starting process, please wait...')
    image_size = slider_image_size.value
    contrast = slider_contrast.value
    threshold = slider_threshold.value
    seed = slider_seed.value
    image_path = Path(image_dir) / dropdown_images.value
    model_path = Path(model_dir) / dropdown_model.value
    model = models.load_model(model_path, backbone_name='resnet50') 

    # Recreate imagelist in case cells are run interactive
    exclude_strings = ['_detected', '_processed']
    imagelist = [i for i in Path(image_path).glob('**/*.jpg') if not any(x in str(i) for x in exclude_strings)]

    np.random.seed(seed)
    image_path = np.random.choice(imagelist, size=1)[0]
    #
    print(f'Selected image: {image_path}')
    # load image
    image = read_image_bgr(image_path)

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = adjust_contrast(image,contrast)
    image = preprocess_image(image)
    image, scale = resize_image(image, min_side=image_size, max_side=2048)

    print('Predicting organoids, please wait...')
    # process image
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))

    # correct for image scale
    boxes /= scale

    num_org = 0
    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < threshold:
            break
        num_org= num_org + 1
        b = box.astype(int)
        draw_box(draw, b, color=(255, 0, 255))

    print(f'{num_org} particles were detected in {image_path}')

    # num_org, cv2.imshow.('test', draw)
    plt.figure(figsize=(25,20))
    plt.imshow(draw)
    plt.title(image_path)
    plt.show()
    
def process_organoids(A):
    outA.clear_output()
    with outA:
        detect_organoids()
 
display(process_button, outA)
process_button.on_click(process_organoids)

Button(button_style='primary', description='Process ', icon='cogs', style=ButtonStyle(), tooltip='Click to tes…

Output()

## 3. Define string regex
Guide: [Python Regex](https://pythex.org/)

Example: '^(?P<WELL>[A-Z]{1}_[0-9]{1,2})_.*_t(?P<T>[0-9]{1,2})_.*\\.jpg$'

Please specify both WELL and T as capture groups

In [15]:
regex_string = widgets.Text(value='^(?P<WELL>[A-Z]{1}_[0-9]{1,2})_.*_t(?P<T>[0-9]{1,2})_.*\.jpg$',
                            placeholder='Type name regex',
                            description='Name regex:',
                            disabled=False
)

display(regex_string)

Text(value='^(?P<WELL>[A-Z]{1}_[0-9]{1,2})_.*_t(?P<T>[0-9]{1,2})_.*\\.jpg$', description='Name regex:', placeh…

In [16]:
outB = widgets.Output()

test_button = widgets.Button(description='Test regex',
                             disabled=False,
                             button_style='primary',
                             tooltip='Click to test regex',
                             icon='cogs' # (FontAwesome names without the `fa-` prefix)
)

def test_regex(B):
    outB.clear_output()
    with outB:
        example = next(Path(Path(image_dir) / dropdown_images.value).glob('**/*.jpg')).name
        print(f'Example file name: {example}')
        try:
            match = re.search(regex_string.value, example)
            WELL = match.group('WELL')
            T = match.group('T')
            print(f'WELL: {WELL}')
            print(f'T: {T}')
            print('Regex succesfull specified!')
            print('Not happy? Change regex and try again...')
            print('Happy? Continue to step 4')
        except:
            print('Incorrect regex specified, please make sure the capture groups WELL and T are present.')

display(test_button, outB)
test_button.on_click(test_regex)

Button(button_style='primary', description='Test regex', icon='cogs', style=ButtonStyle(), tooltip='Click to t…

Output()

## 4. Save config
Happy with the result? Save config and you are done!

In [17]:
save_button = widgets.Button(
    description='Save config',
    disabled=False,
    button_style='success',
    tooltip='Click to save config',
    icon='save' # (FontAwesome names without the `fa-` prefix)
)

outC = widgets.Output()

def save_config(C):
    outC.clear_output()
    with outC:
        config = {'model': str(Path(model_dir) / dropdown_model.value),
                  'image_size': slider_image_size.value,
                  'contrast': slider_contrast.value,
                  'threshold': slider_threshold.value,
                  'regex': regex_string.value}
  
        with open(f'{Path(image_dir) / dropdown_images.value}/config.json', 'w') as fp:
            json.dump(config, fp)

        print('Save Succesfull!')
        print('Saved json:')
        print(config)

display(save_button, outC)
save_button.on_click(save_config)

Button(button_style='success', description='Save config', icon='save', style=ButtonStyle(), tooltip='Click to …

Output()

## 5. Close notebook
Please close the notebook